<a href="https://colab.research.google.com/github/Saloni043/FMML-LABS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits does tend to give more consistent results. This approach is a common practice in machine learning and is generally referred to as cross-validation. Here's why it helps:

1. Reduces Variance

Single Train-Test Split:

If you evaluate your model using a single train-test split, the results can be highly sensitive to the specific data points chosen for training and testing. This can lead to high variance in your performance metrics, especially if your dataset is small or unbalanced.


Multiple Splits (Cross-Validation):

By splitting the data into multiple training and testing subsets (e.g., in K-fold cross-validation), the model is trained and evaluated multiple times, with each split providing a different perspective on how the model performs.

The average validation accuracy across these splits provides a more reliable estimate of model performance, smoothing out the effects of any one particular data split.



2. More Robust Performance Estimation

Generalization:

Averaging over multiple folds helps the model's evaluation account for variability in the data, leading to a better understanding of how it will perform on unseen data. This is because each fold serves as a test set at least once, providing a broader picture of performance.


Avoiding Overfitting:

Cross-validation helps reduce overfitting by ensuring the model is tested on data it hasn't seen during training. If you rely on a single split, the model might "memorize" that particular training set and perform poorly on unseen data.



3. Cross-Validation Methods

K-Fold Cross-Validation:

The data is split into K subsets (or "folds"). The model is trained K times, each time using K-1 folds for training and the remaining fold for testing. The validation accuracy is averaged over the K iterations.


Stratified K-Fold Cross-Validation:

This is similar to K-fold but ensures that each fold has the same proportion of each class as the original dataset. It is particularly useful when dealing with imbalanced class distributions.


Leave-One-Out Cross-Validation (LOO-CV):

A special case of cross-validation where K is equal to the number of data points. Each instance is used once as the validation set while the remaining data points are used for training. This method is computationally expensive but provides very reliable results, especially for small datasets.



4. Example

If you perform 5-fold cross-validation:

The dataset is divided into 5 folds.

The model is trained on 4 folds and validated on the remaining fold.

This process is repeated 5 times, each time with a different fold as the validation set.

You then average the accuracy scores from each fold to get a final estimate of the model's performance.



---

5. Impact on Consistency

Single Split: You might get a validation accuracy of 75%, but this could be due to the particular data points in the split.

Multiple Splits: By averaging across, say, 10 folds, the variance of the accuracy estimate decreases. The resulting value is typically more stable and representative of how the model will perform across the entire dataset.



---

Summary

Averaging the validation accuracy across multiple splits reduces the impact of random fluctuations in the data, providing a more consistent and reliable estimate of model performance.

It also helps to evaluate how well the model generalizes to unseen data.


Would you like an example of how to implement cross-validation in Python?




2. Does it give more accurate estimate of test accuracy?
Averaging the validation accuracy across multiple splits in cross-validation provides a more accurate estimate of the model's performance on unseen data, but it does not directly give the test accuracy. Here's why:

1. Cross-Validation vs. Test Set Accuracy

Cross-Validation:

Cross-validation helps estimate how well the model is likely to perform on unseen data by training and validating the model on different subsets of the data.

The validation accuracy from cross-validation provides an estimate of the model's ability to generalize, but it does not directly correspond to the accuracy on a completely separate test set that the model has never seen before.


Test Set Accuracy:

Test accuracy refers to the performance of the model on a separate, hold-out test set that was never part of the training or validation process. This is the final evaluation of how well the model generalizes to new, unseen data.

If you use cross-validation and average the validation accuracies, this gives you an estimate of how well the model will perform on new data, but it doesn't guarantee that the test accuracy (on a truly separate test set) will be the same.



2. Relationship Between Cross-Validation and Test Accuracy

Improved Generalization Estimate:

Cross-validation provides a better generalization estimate than using a single train-test split. It reduces the variance of your performance estimate and gives a more reliable prediction of how the model will perform on new, unseen data, which is often similar to the test set.


Bias and Variance:

Cross-validation helps reduce bias (by training the model on different data subsets) and variance (by averaging across different splits), giving a less biased and more consistent estimate of generalization.

However, this is still an estimate, and there may be some difference between cross-validation performance and the performance on a separate test set. Cross-validation performance is subject to data distribution, and the performance on the test set may differ due to different samples or data characteristics.



3. Overfitting and Model Selection

When you train a model and perform cross-validation, the goal is often to choose the best model or hyperparameters. Once the model is selected, it should be evaluated on a hold-out test set to check for overfitting and assess its true performance on unseen data.

Cross-validation can help ensure that the model has not been overfitted to the training data, but the final test accuracy provides the most definitive measure of the model's ability to generalize.


4. How Cross-Validation Impacts Model Tuning

Hyperparameter Tuning: Cross-validation is typically used for selecting hyperparameters (like learning rate, regularization strength, etc.). Once hyperparameters are chosen, a final model is trained on the entire training set, and the performance on a separate test set is reported.

Multiple Splits: If you perform K-fold cross-validation, you're effectively using more data for both training and validation, which typically results in a better estimate of the generalization error than a single split.


5. Final Test Set Evaluation

After Cross-Validation: Once you've used cross-validation for model selection, you should still evaluate the final model on a separate test set (one that was not used during cross-validation) to get an unbiased estimate of test accuracy.



---

Summary

Averaging validation accuracies across multiple splits in cross-validation provides a more reliable and consistent estimate of how the model will generalize to unseen data.

While cross-validation gives a good approximation of the model's ability to perform on new data, it does not directly provide the test accuracy. The test accuracy is still the final measure of model performance, and it should be assessed on a completely separate hold-out test set after model selection and training.


Would you like an example to implement both cross-validation and final test evaluation in Python?


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Yes, increasing the number of iterations (or the number of splits, if using K-fold cross-validation) can improve the accuracy and reliability of the estimate of model performance. However, the effect depends on the context, and there's a balance between computation time and the quality of the estimate. Here's a more detailed explanation:

1. Effect of the Number of Iterations (or K in K-Fold Cross-Validation)

More Iterations (More Folds in Cross-Validation):

With higher iterations (i.e., more folds in cross-validation), each data point gets to be used for validation multiple times, giving a better understanding of how the model performs on different subsets of the data.

Larger K (e.g., 10-fold vs. 5-fold):

Provides a more robust estimate of the model's generalization performance.

Each fold has fewer training samples, but more data points are used for validation, which can lead to better insights into how the model generalizes across the entire dataset.




2. Diminishing Returns

While more iterations lead to a more accurate and stable estimate, there are diminishing returns after a certain point:

After a certain number of folds (usually after K=10 or 20), the improvement in the estimate becomes minimal.

For example:

With 5-fold cross-validation, the model is validated on 20% of the data in each fold. Increasing to 10 folds gives a slight improvement because 10% of the data is used for validation, offering a slightly better idea of performance.

Beyond a certain point (e.g., 10 or 20 folds), the added benefit becomes marginal because the model has already been evaluated on a substantial portion of the data.




3. Tradeoff Between Accuracy and Computation

Higher K (more folds/iterations) means more training and evaluation steps:

Computation time increases linearly with K. For example, in 10-fold cross-validation, the model will be trained 10 times, which can be computationally expensive, especially with large datasets or complex models.

Time and Resources: The more iterations you use, the longer it takes to complete the cross-validation process, so there's a trade-off between better estimates and computational cost.


Practical Range:

For most models, 10-fold cross-validation is a commonly used and generally sufficient choice.

Leave-One-Out Cross-Validation (LOO-CV) has the maximum number of iterations (equal to the number of data points) but is rarely used for large datasets because of the high computational cost.



4. Stability of the Estimate

Higher iterations lead to a more stable and reliable estimate of model performance:

When you average performance over more iterations, the estimate becomes less sensitive to the peculiarities of any single data split.

Example: If you use only a single split or a small number of folds, you might get an accuracy of 70% on one split and 80% on another. But with more iterations, the variability reduces, and you get a more consistent estimate (e.g., 75% on average).



5. Bias vs. Variance

Bias: The more iterations you run, the more representative your validation score will be of the model's generalization ability.

Variance: With fewer iterations (or folds), the estimate is more likely to vary because you're testing on only a few subsets of data. More iterations smooth out these fluctuations.



---

6. Effect of High Iterations on the Estimate

Better Estimate: With a higher number of iterations (e.g., 20-fold or LOO-CV), you get a more accurate estimate of the model's true generalization performance.

Example: If you use a 5-fold cross-validation and the estimated validation accuracy is 80%, increasing to 10-fold might show 81%, and increasing to 20-fold might show 81.5%, providing a more precise estimate.


Real-World Use:

If you're looking for very reliable estimates, such as in cases with smaller datasets, using higher iterations can make sense. For larger datasets, the benefit might be marginal.




---

7. Computational Cost Considerations

Increasing the number of folds or iterations directly impacts the training time:

For each fold, the model must be trained and validated, meaning the computational cost increases linearly with the number of iterations.

For very large datasets or complex models, 10-fold cross-validation might be a sweet spot, offering good balance between computational cost and estimate accuracy.




---

8. Practical Guidelines

10-fold cross-validation is typically the standard in most machine learning tasks and often provides a good trade-off between computation time and reliability of the estimate.

For small datasets, you might opt for Leave-One-Out Cross-Validation (LOO-CV) for a more precise estimate, though it can be computationally expensive.

For large datasets, 5-fold cross-validation might be sufficient.



---

Summary

Higher iterations (more folds) in cross-validation lead to better, more consistent estimates of the model’s generalization performance because they reduce variability and bias.

However, there are diminishing returns beyond a certain point (e.g., K=10), and it becomes computationally expensive to go much higher.

The optimal number of iterations depends on the dataset size, the model's complexity, and the computational resources available




4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Increasing the number of iterations (or folds in cross-validation) can help mitigate some of the challenges when you have a small training or validation dataset, but it does not completely solve the issues associated with small datasets. Let's break this down:

1. Effect of Small Datasets

When you have a small dataset:

Limited Data for Training: The model has fewer examples to learn from, which can lead to overfitting or poor generalization.

Limited Data for Validation: A small validation set may not adequately represent the diversity of the data, leading to high variance in model evaluation and less reliable performance estimates.

High Variance: The results might vary significantly between different splits of the data because each split is small and might not capture the full variability of the data.


2. Can Increasing Iterations Help?

Increasing the number of iterations or folds in cross-validation can reduce the variance in the performance estimates and give a better sense of how the model generalizes, but there are some key points to consider:

Pros of Increasing Iterations for Small Datasets:

More Efficient Use of Data: By using multiple folds, more data is used for both training and validation across different iterations. This allows the model to "see" more of the data for training and testing, which can help improve generalization.

Example: If you have only 20 samples and use 5-fold cross-validation, each fold will use 16 samples for training and 4 for validation. If you use 10-fold cross-validation, each fold will use 18 samples for training and 2 for validation, increasing the number of training samples for each fold.


More Robust Performance Estimate: Averaging the results over more folds helps reduce the influence of outliers or variability in the small validation sets. This can lead to a more reliable estimate of how the model is likely to perform on new data.


Cons or Limitations:

Still Small Validation Data: Even with more iterations, each individual fold will still be based on a small portion of the dataset, meaning that the validation set may not be sufficiently representative of the entire data distribution.

For instance, in Leave-One-Out Cross-Validation (LOO-CV), even though you use all but one data point for training, the single validation point might not capture the full variability in the data.


Training Data Limitations: Increasing the number of folds means that each training set has fewer data points (e.g., in 10-fold cross-validation, each training set is 90% of the data, and the model will see less data per fold compared to a smaller number of folds). This might make it harder for the model to learn effectively, especially for complex models.

Risk of Overfitting: With small datasets, increasing the number of folds can sometimes exacerbate overfitting. Each model is trained on slightly different subsets of data, and if the data is very small, the model might overfit to specific patterns seen in individual folds, leading to high variance across the folds.


3. Alternatives to Increasing Iterations for Small Datasets

While increasing iterations in cross-validation can help somewhat, there are better alternatives to address the challenges of small datasets:

Data Augmentation: If applicable, you can use data augmentation techniques (e.g., for image data, you can rotate, flip, or scale images) to artificially increase the size of the dataset.

Synthetic Data Generation: Methods like SMOTE (Synthetic Minority Over-sampling Technique) can be used to generate synthetic examples, especially when dealing with imbalanced datasets.

Transfer Learning: For small datasets, using a model that has been pre-trained on a larger dataset and fine-tuning it on your smaller dataset (common in image and NLP tasks) can be highly effective in improving generalization.

Regularization: Using regularization techniques (e.g., L2 regularization or dropout in neural networks) can help prevent overfitting by discouraging overly complex models that may fit to the noise in a small dataset.

Bootstrap Aggregating (Bagging): Instead of cross-validation, you can use bootstrapping (sampling with replacement) to generate new datasets for training. This can help stabilize the model by averaging results over multiple resamples.

Bayesian Methods: For small datasets, Bayesian methods can provide better uncertainty estimates, incorporating prior knowledge about the model and data.


4. Practical Considerations

Very Small Datasets: If the dataset is extremely small (e.g., fewer than 20-30 samples), cross-validation (even with many iterations) may still struggle to provide reliable performance estimates. In such cases, it’s often better to explore alternative models or techniques that can handle small data more effectively, such as few-shot learning or transfer learning.

Computational Efficiency: If the dataset is small, using too many folds (e.g., LOO-CV) can be computationally expensive and may not provide significant improvements in performance estimates compared to fewer folds.



---

Summary

Increasing iterations (folds) in cross-validation can improve the reliability and consistency of performance estimates, especially with small datasets, by reducing the variance in validation results.

However, increasing iterations alone will not completely solve the problems associated with small datasets (e.g., high variance, limited data for training and validation).

Alternative strategies, like data augmentation, transfer learning, or regularization, are often more effective for improving model performance on small datasets.


Would you like to explore any of these alternative strategies further, or see examples for handling small datasets?

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.